In [1]:
import os
from malaria_cnn import logger

In [2]:
os.chdir('../')
%pwd

'/home/fibonacci/projects/Detecting_Malaria_CNN'

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_model_path:Path
    Training_data:Path
    params_epochs:int
    params_batch_size:int
    params_is_augmentation: bool
    params_image_size:list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [4]:
from malaria_cnn.constant import *
import tensorflow as tf
from malaria_cnn.utils.common import read_yaml,create_directories

2024-05-06 08:44:42.247558: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-06 08:44:47,174: NumExpr defaulting to 4 threads.: INFO


In [5]:
##configuration manager
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        prepare_callback_config=PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config
    
    def get_training_config(self)-> TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"cell_images/cell_images"
                                   )
        create_directories([
            Path(training.root_dir)
        ])
        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_model_path=Path(prepare_base_model.updated_model_path),
            Training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config


In [6]:
import time

In [7]:
class PrepareCallbacks:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config=config
    
    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_dir=os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",      
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_weights_only=True,
            save_best_only=True
        )
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [8]:

import os
import urllib.request as request
import zipfile
import tensorflow as tf
import time



In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None  # Initialize model attribute

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.2
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.Training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )
            self.train_generator = train_datagenerator.flow_from_directory(
                directory=self.config.Training_data,
                subset='training',
                shuffle=True,
                **dataflow_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.Training_data,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        if self.model is None:
            self.get_base_model()  # Ensure model is loaded before training

        self.train_valid_generator()  # Ensure generators are initialized

        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Recreate optimizer to ensure compatibility with the new model
        optimizer = tf.keras.optimizers.Adam()  # Modify with your optimizer and settings

        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        self.model.fit(
            self.train_generator,
            steps_per_epoch=self.steps_per_epoch,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callback_list
        )

        self.save_model(path=self.config.trained_model_path, model=self.model)


In [12]:
try:
    config=ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callbacks_config()
    prepare_callbacks=PrepareCallbacks(config=prepare_callbacks_config)
    callbacks_list=prepare_callbacks.get_tb_ckpt_callbacks()

    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callbacks_list)

except Exception as e:
    logger.exception(e)

2024-05-06 08:49:30,383: yaml file:config/config.yaml load successfully: INFO
2024-05-06 08:49:30,387: yaml file:params.yaml load successfully: INFO
2024-05-06 08:49:30,390: Directory artifacts created successfully: INFO
2024-05-06 08:49:30,392: Directory artifacts/prepare_callbacks/checkpoint_dir created successfully: INFO
2024-05-06 08:49:30,394: Directory artifacts/prepare_callbacks/tensorboard_log_dir created successfully: INFO
2024-05-06 08:49:30,398: Directory artifacts/training created successfully: INFO
2024-05-06 08:49:30,764: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.: WARNING


Found 5510 images belonging to 2 classes.
Found 22048 images belonging to 2 classes.
Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.
Found 22048 images belonging to 2 classes.
Found 22048 images belonging to 2 classes.
Epoch 1/10


/home/fibonacci/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-05-06 08:49:41.151309: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-05-06 08:49:43.137013: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-05-06 08:49:46.967210: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2024-05-06 08:49:47.031352: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-05-06 08:49:47.89

 112/1378 ━━━━━━━━━━━━━━━━━━━━ 6:47:04 19s/step - accuracy: 0.6221 - loss: 0.7481